# Classical Mechanics I (PHYS 311)
## Studio 9

*Name:* Connor Haskell

*Date:* 10/30/2024

## More complex animations

Make sure you were able to do everything in Studio 8!

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Remember we have these magic lines to give us access to the animation and conversion functionalities.

In [2]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

#Last time, we defined a function `plotSomeStuff`.


def plotSomeStuff(x_array, y_array, nframes=100, ax = None, fig = None):
    if ax == None or fig == None:
        fig, ax = plt.subplots()
        ax.set_xlim(-10,10)
        ax.set_ylim(-10,10)

    phi = linspace(0,2*np.pi,100)

    marker1, = ax.plot([],"o") # Let's create an empty plot that has a circle marker
    orbit1, = ax.plot([]) # Let's create an empty plot to eventually become an orbit line

    def animate(frame_num):
        """
        """
        x = x_array[frame_num]
        y = y_array[frame_num]
        
        # Let's put a marker at the last position of these arrays.
        # We're using set_data which is updating the data stored in the marker1 object.
        marker1.set_data([x,y])
        orbit1.set_data([ x_array[:frame_num],y_array[:frame_num] ])
        return

    anim = FuncAnimation(fig, animate, frames=nframes, interval=20)
    display(HTML(anim.to_jshtml()))
    plt.close()




#Let's now write a new function based on this. Instead of the first two arguments being numpy arrays, let's make them lists of numpy arrays. So instead of the first argument being `x_array` of type `np.array`, let's have it be `x_arrays` which looks like `[np.array, np.array, ...]`. And then have it animate the trajectories of all of the pairs of arrays handed to the function.

In [19]:
def plotSomeStuff(x_arrays, y_arrays, nframes=100, ax = None, fig = None):
    if ax == None or fig == None:
        fig, ax = plt.subplots()
        ax.set_xlim(-10,10)
        ax.set_ylim(-10,10)

        
    marker = {}
    orbit = {}
    
    for ipath,(x_array, y_array) in enumerate(zip(x_arrays,y_arrays) ):
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!
        # This is an example of how to loop over all pairs of x and y arrays.
        # The zip function takes two lists and combines them into a single list like:
        # zip( [a, b], [c, d] ) will give [ [a,c], [b,d] ]
        # The enumerate function gives you an index for each object in an iterable list.
        # So enumerate([a,b]) will give a new list [ [0,a], [1,b] ]
        marker[ipath], = ax.plot([],"o")
        orbit[ipath], = ax.plot([])

    def animate(frame_num):
        for ipath, (x_array, y_array) in enumerate(zip(x_arrays, y_arrays)):
            # Update the position of the marker and the orbit for each frame
            marker[ipath].set_data(x_array[frame_num], y_array[frame_num])
            orbit[ipath].set_data(x_array[:frame_num], y_array[:frame_num])
        return list(marker.values()) + list(orbit.values())
    
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!
        # Now write some code here that would loop over all pairs of the x and y arrays (see above)
        # and then use the set_data function like last week to edit each object in marker and orbit.
        
        return

    anim = FuncAnimation(fig, animate, frames=nframes, interval=20)
    display(HTML(anim.to_jshtml()))
    plt.close()

# Create circular orbits for two particles
theta = np.linspace(0, 2 * np.pi, 100)
r1, r2 = 5, 3  # Different radii for the two particles

x_array1 = r1 * np.cos(theta)
y_array1 = r1 * np.sin(theta)

x_array2 = r2 * np.cos(-theta)  # Negative sign to go in the opposite direction
y_array2 = r2 * np.sin(-theta)

# Run the function
# !!!!!!!!!!!!!!!!!!!!
# Construct the x and y arrays for two particle circular orbits (of differing radius)
# going in opposite directions
# Store them as x_array1, y_array1 (to describe the path of particle 1)
#           and x_array2, y_array2 (to describe particle 2's path)
nframes = 100
# Now we can run the function
plotSomeStuff([x_array1,x_array2],[y_array1,y_array2],nframes)

## Two-body problem

Now we have a way of plotting the trajectories of multiple particles. Last week, we solved a system of ODEs to describe the motion of a single particle. Recall that we could map the 2-body gravitational problem onto a single particle of "reduced" mass $\mu$ in a gravitational well centered at $\vec{r}=0$. So let's remember how to get back to two particles from this simplified 1D problem.

$$\mu=\frac{m_1m_2}{M}$$

$$\vec{r}_1 = \vec{R}+\frac{m_2}{M}\vec{r}$$

$$\vec{r}_2 = \vec{R}-\frac{m_1}{M}\vec{r}$$

Let's set $\vec{R}$ to 0 such that we're in the system's CoM frame at all times. This gives us all the ingredients we need.

Solve the same ODE as last week, which will give you the x and y components of the vector $\vec{r}$ as a function of time. Then convert those solutions into two separate vectors $\vec{r}_1$ and $\vec{r}_2$. Then hand those to your multi-particle plotting script above. You'll need to define values for the masses of your two particles, then calculate $\mu$, and make sure you hand it to your ode_system call (using the `args=(thing,)` in your `odeint` call).

Let's start with $m_1=m_2$.

In [4]:
from scipy.integrate import odeint

In [20]:

def ode_system(inputs,t,m):
    """
    This function represents a series of first order ODEs.
    
    Return: List of expressions for the first time derivative of the inputs, in order.
    """
    
    # Parse the inputs list to positions x,y and vector magnitudes xdot,ydot
    [x,y,xdot,ydot] = inputs
    
    r = np.hypot(x,y) # This is a handy function that computes sqrt(x^2+y^2) for you
    phi = np.arctan2(y,x) # And this calculates the polar angle of our vector

    Fr = -1/(r*r)
    Fx = Fr*np.cos(phi)
    Fy = Fr*np.sin(phi)
    
    # Return the expressions for the time-derivatives of the inputs.
    return [xdot,ydot,Fx/m,Fy/m]


# solve the ODE for position r of mass mu.
# Convert the result in terms of r1 and r2. Plot these particle paths.

# Initial conditions: [x0, y0, xdot0, ydot0]
initial_conditions = [5, 1, 0, 0.5]  # Starting position and initial velocity

# Masses of the particles
m1 = 1.0
m2 = 5.0
M = m1 + m2  # Total mass
mu = (m1 * m2) / M  # Reduced mass

# Time array for the solution
t = np.linspace(0, 150, 100)

# Solve the ODE for the reduced mass
solution = odeint(ode_system, initial_conditions, t, args=(mu,))

# Extract the x and y positions of the reduced mass
x_sol = solution[:, 0]
y_sol = solution[:, 1]

# Convert to the positions of the two particles
x1_sol = (m2 / M) * x_sol
y1_sol = (m2 / M) * y_sol
x2_sol = -(m1 / M) * x_sol
y2_sol = -(m1 / M) * y_sol


plotSomeStuff([x1_sol, x2_sol], [y1_sol, y2_sol], nframes=len(t))

Play with your initial conditions to find some bound and unbound 2-body orbits. If you think you have an unbound orbit, make sure you let your time array go to large enough values to make sure they don't later come back!

- The intial distance between the particle effects the boundess of the orbit. Closer initial distance will lead to tighter orbits and far intial distances will lead to bigger eliptical orbits and higher chance of having unbound orbit.
- If the intial velocity is below the escape velocity of the system, then the result is bound orbit, when equal to esc v we get parabloc, and when greater than esc v we get hyperbolic or unbound trajectory.
- If the masses are equal, then the mass paths will be more symmetric with each other - they orbit around the CoM equally.
- With one masss being larger, the larger mass's path will stay closer to the CoM 
- An increase in one mass will increase the gravitational pull between the masses.


Now start to play with the masses to see how the situation changes! The orbits change significantly!

**EC(+0.5): Define a new plotting function which can take the marker sizes as arguments, so that each circle size can be controlled individually. Then plot a 2-body problem orbit with two different mass values, and have the area of the circle be proportional to the mass of the particle (remember that the `plot` marker size changes the diameter of the circle).**

And that's it! You've solved the 2-body problem!